In [7]:
from pymongo import MongoClient
import pymongo
import datetime
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm

In [2]:
client = MongoClient(host='114.70.235.37', port=27017)
print(client.list_database_names())

['admin', 'config', 'local', 'mydb']


In [3]:
db = client['mydb'].xy
result= db.find().sort([("date", pymongo.DESCENDING)]).limit(2)

data = list(result)[0]
start_date = data["date"] - datetime.timedelta(seconds=5)
seq = list(db.find({"date": {'$gte' : start_date, '$lt' : data["date"]}}))

In [4]:
list_data = list()
for j in seq:
    del j['date']
    del j['_id']
    
    list_data.append(list(j.values()))

In [5]:
list_data = np.array(list_data, dtype=np.float32)

In [14]:
y_false = np.array([0 for w in range(len(list_data))])
print(list_data.shape)
print(y_false.shape)

(183, 20)
(183,)


In [8]:
# dataloaders
batch_size = 32
device='cuda'

class SentimentLSTM(nn.Module):

    def __init__(self, seq_size, output_size, hidden_dim, n_layers):
        
        super(SentimentLSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # embedding and LSTM layers
        self.lstm = nn.LSTM(seq_size, hidden_dim, n_layers, batch_first=True, dropout=0.2)
        
        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        batch_size = x.size(0)
        
        lstm_out, hidden = self.lstm(x)
        ## lstm_out.shape [batch, seq_len, hidden_dim]
        ## hidden [batch, hidden_dim]

        # fully connected layer        
        out = self.fc(lstm_out[:, -1, :])
        ## out.shape: [n_layer * n_direction, batch, output_size]

        # sigmoid function
        sig_out = self.sig(out)
        
        # return last sigmoid output and hidden state
        return sig_out, out

In [9]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        # nn.init.uniform_(param.data, -0.1, 0.1)

In [10]:
hidden_dim = 512
n_layers = 4
lr = 0.01
output_size = 1

model = SentimentLSTM(20, output_size, hidden_dim, n_layers)
model.to('cuda')

SentimentLSTM(
  (lstm): LSTM(20, 512, num_layers=4, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [11]:
model.load_state_dict(torch.load("model_weight.pth"))

<All keys matched successfully>

In [12]:
def predict(model, test_loader, sequence_length=30):
    
    model.eval()
    
    batch_size = 32
    acc_list = list()
    
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output, h = model(inputs)
        
        pred = torch.round(output.squeeze())
        
        accuracy = (pred==labels).sum().item() / 32
        acc_list.append(accuracy)

        #print('Prediction value, pre-rounding: ', output.item())

        # print custom response based on whether test_review is pos/neg
        
    print(np.mean(acc_list))

In [16]:
tmp_data = TensorDataset(torch.from_numpy(list_data), torch.from_numpy(y_false))
tmp_loader = DataLoader(tmp_data, shuffle=False, batch_size=batch_size)

In [17]:
predict(model, tmp_loader)

IndexError: too many indices for tensor of dimension 2

In [18]:
tmp_data[0]

(tensor([ -22.9623,  -37.8234, -118.5906, -235.3448,   22.9623,   37.8234,
          -85.7970,  -33.0598,    0.0000,    0.0000,  -20.1370, -140.5871,
           43.0994,  178.4105,   43.0994,  178.4105,    0.0000,    0.0000,
            0.0000,    0.0000]),
 tensor(0))